In [2]:
from collections import defaultdict 
from sklearn.cluster import KMeans
import numpy as np
import random
import copy

# node class - an item is created for each sensor node in a given network
class Node():

    def __init__(self, id, energy, location):
        self.id = id
        self.energy = energy
        self.location = location
        self.routing_tbl = [[]] * 4
        self.hubID = -1

    def setTransmitPwr(self, transmit_pwr):
        self.transmit_pwr = transmit_pwr

    def setProcPower(self, processing_pwr):
        self.processing_pwr = processing_pwr

    def setHub(self, hubID):
        self.hubID = hubID

    def addRoute(self, var, index):
        # self.routing_tbl.append(route)
        if (index == 0):
            # THIS IS CHANGING ROUTING NUMBER
            self.routing_tbl[0] = 1
        elif (index == 1): # these indices are for holding paths
            self.routing_tbl[1] = var
        elif (index == 2):
            self.routing_tbl[2] = var
        elif (index == 3):
            self.routing_tbl[3] = var

# graph class to help when a node sends out an RREQ
class Graph(): 

    def __init__(self, vertices): 
        self.V = vertices
        self.graph = defaultdict(list)

    # adds edge to graph (path from node u to node v)
    def addEdge(self, u, v):
        self.graph[u].append(v)

    # getAllPaths helper function
    def getAllPathsUtil(self, u, d, visited, path):

        visited[u] = True
        path.append(u)
        
        # If node equals destination
        if u == d:
            allPossiblePaths.append(copy.copy(path))
        else:
            for i in self.graph[u]:
                if visited[i] == False:
                    self.getAllPathsUtil(i, d, visited, path)

        path.pop()
        visited[u] = False
    
    # returns all paths from src to dest
    def getAllPaths(self, s, d):
        visited = [False] * (self.V)

        path = []

        self.getAllPathsUtil(s, d, visited, path)

# helper function that calculates distance between two points of dimension d
def dist(p1, p2, d): 
      
    x = []
    for i in range(d):
        x.append(p1[i] - p2[i])
        
    tot = 0
    for i in range(d):
        tot = tot + (x[i] * x[i])
        
    return tot

# takes in nodes, dest, and k (num of clusters); returns network 'layout'
# can use this function to update/change the layout as energy depletes
# or transmission power changes greatly because one of the nodes died/ran out of energy
def layout(nodes, dest, k):
    
    # holds location of all nodes in the network
    node_locs = np.zeros(shape=(len(nodes)+1, 2))
    for n in range(len(nodes)):
        node_locs[n] = nodes[n].location
    # destination location is added as well
    node_locs[len(nodes)] = dest
    
    # use kmeans clustering algorithm in order to split network into smaller groups
    kmeans = KMeans(n_clusters=k).fit(node_locs)
    clusters = []
    for i in range(k):
        clusters.append([])
    # store clusters in a list
    for l in range(len(kmeans.labels_)):
        for i in range(k):
            if kmeans.labels_[l] == i:
                clusters[i].append(node_locs[l])
    
    # go through each cluster and pick a node to be the 'hub', therefore creating
    # a star-mesh network hybrid
    for c in clusters:
        h = c[0] # initialize h with possible hub for cluster c
        proximity = 200
        for i in range(len(c)):
            temp = 0
            for j in range(len(c)):
                temp += dist(c[i], c[j], 2)
            for n in nodes:
                if np.all(n.location == c[i]):
                    # transmission power is dependent on proximity of node to others
                    n.setTransmitPwr(float(temp) / float(len(nodes)))
                    # processing power is a fraction of the transmission power
                    n.setProcPower(n.transmit_pwr / 5)
            # the node with closest proximity to all other nodes in its cluster
            # will become the hub since that means it'll use the least amount of 
            # transmission power and will conserve overall cluster energy for longer
            if temp < proximity:
                proximity = temp
                for n in nodes:
                    if np.all(n.location == c[i]):
                        
                        h = n
        # set hub id for all nodes in the cluster
        for l in c:
            for n in nodes:
                if np.all(n.location == l):
                    n.setHub(h.id)

# 'runs' the network / starts sending packets
def sendPacket(nodes, src, dest, packet_size):

    packets = 0 # var for how many packets were sent before failure
    packet_size = 512 # let's say all packets being sent are of size 512 B
    
    hubs = [nodes[0].hubID]
    for n in nodes:
        if n.hubID not in hubs:
            hubs.append(n.hubID)
    for h in range(len(hubs)):
        hubs[h] = nodes[hubs[h]]

    # there is only one hub so it's the only one that can go to 
    # destination, all the other nodes can only send info to hub
    if len(hubs) == 1:
        hub = hubs[0]
        hub.energy -= (packet_size * (hub.transmit_pwr / 1000))
        if src != hub.id:
            # update energy left after packet is sent
            nodes[src].energy -= (packet_size * (nodes[src].transmit_pwr / 1000))
            # make sure there was enough energy to send it
            if nodes[src].energy >= 0 and hub.energy >= 0:
                packets +=1 # energy left >= 0 so packet was sent successfully
            else:
                nodes[src].energy += (packet_size * (nodes[src].transmit_pwr / 1000))
                hub.energy += (packet_size * (hub.transmit_pwr / 1000))
        else:
            if hub.energy >= 0:
                packets += 1
            else:
                hub.energy += (packet_size * (hub.transmit_pwr / 1000))
    # more than one hub
    else:
        # first, check routing table for a path
        bestPath = []
        bp = nodes[nodes[src].hubID].routing_tbl[3]
        if len(bp) > 0:
            v = True
            # make sure path is viable
            for n in bp:
                if n != -1:
                    if nodes[n].energy < (packet_size * (nodes[n].transmit_pwr / 1000)):
                        v = False
            if v:
                bestPath = bp
                
        # if no viable bestPath was found in routing table        
        if len(bestPath) == 0:
            g = Graph(len(nodes)+1)
            for h in range(len(hubs)):
                for h2 in range(len(hubs)):
                    if h != h2:
                        g.addEdge(hubs[h].id, hubs[h2].id)
                g.addEdge(hubs[h].id, -1) # don't forget to add destination node

            # variable to hold all found paths from src's hub to dest        
            global allPossiblePaths 
            allPossiblePaths = []

            # If src equals the hub
            if src == nodes[src].hubID:
                g.getAllPaths(src, -1)
            else:
                g.getAllPaths(nodes[src].hubID, -1)

            # make sure paths are viable
            for p in allPossiblePaths:
                v = True
                for n in p:
                    if n != -1:
                        if nodes[n].energy < (packet_size * (nodes[n].transmit_pwr / 1000)):
                            v = False
                if not v:
                    allPossiblePaths.remove(p)

            # calculate cost of each possible path
            allPathCosts = []
            for path in allPossiblePaths:
                path_cost = 0
                for p in path:
                    # if destination not reached
                    if p != -1:
                        # continue to add costs - implement processesing power later
                        path_cost += (packet_size * (nodes[p].transmit_pwr / 1000))
                allPathCosts.append(path_cost)

            # min path cost will be our chosen path
            bestPath = allPossiblePaths[allPathCosts.index(min(allPathCosts))]

            # Optimal path from a hub to destination, when multiple hubs
            # Should loop for all hubs?
            nodes[nodes[src].hubID].addRoute(bestPath, 3)
        
            # calculate hops
            if src == nodes[src].hubID: # if src is a hub
                hops = len(bestPath)
            else:
                hops = len(bestPath) + 1 # add one because src must hop to hub

            # Adding the number of hops to the table
            nodes[nodes[src].hubID].addRoute(hops, 1)
            # Also adding the cost of the route to the table
            nodes[nodes[src].hubID].addRoute(min(allPathCosts), 2)
        
        # apply costs of path and make sure packet is sent
        success = configureMultiNodePath(nodes, src, bestPath, packet_size)
        
        if success:
            packets += 1

    return packets

# takes in nodes, src node, and best path (path to be executed) - applies transmission
# costs and updates node energies - returns whether packet was successful in being sent
def configureMultiNodePath(nodes, src, path, packet_size):

    packet_sent = True
    
    # holds all nodes in path / used to send packet
    iterated_nodes = []
    
    # If the src node is NOT a hub, apply cost to src node energy
    if src != nodes[src].hubID:
        nodes[src].energy -= (packet_size * (nodes[src].transmit_pwr / 1000))
        iterated_nodes.append(src)
        if nodes[src].energy < 0:
            packet_sent = False
    
    # apply costs to nodes along chosen path
    for p in path:
        if p != -1: # make sure we're not trying to apply costs to destination
            nodes[p].energy -= (packet_size * (nodes[p].transmit_pwr / 1000))
            iterated_nodes.append(p)
            if nodes[p].energy < 0:
                packet_sent = False
    
    # if one of the nodes didn't have enough energy to send the packet, costs
    # are reversed and sending of packet was unsuccessful
    if not packet_sent:
        for i in iterated_nodes:
            nodes[i].energy += (packet_size * (nodes[i].transmit_pwr / 1000))
    
    return packet_sent
    
# checks if network is still capable of sending packets
def checkNodeEnergy(nodes, packet_size):
    
    live = False
    
    counter = 0
    for n in nodes:
        if n.energy >= (packet_size * (n.transmit_pwr / 1000)):
            counter += 1
    
    # at least one node has enough energy left to send a packet
    if (counter > 0):
        live = True

    return live

# checks to see if any of the hubs are running low on energy and therefore if the 
# layout/organization of the network needs to be updated
def checkHubsAvailability(nodes, packet_size, init_energy):
    
    # store all hubs in a list
    known_hubs = []
    for n in nodes:
        if n.hubID not in known_hubs:
            known_hubs.append(n.hubID)
    
    # get a list of the hubs that are low energy
    low_hubs = []
    for h in known_hubs:
        # energy is considered low when it's less than initial energy * cost of 
        # sending one packet
        if nodes[h].energy < (init_energy * packet_size * nodes[h].transmit_pwr / 1000):
            low_hubs.append(h)
    
    return low_hubs
    
    
# takes in low energy hubs so that the layout/organization of the network
# can be updated accordingly
def changeHubs(nodes, hubsToBeChanged, init_energy):
    
    new_hubs = []
    
    updated_hubs = []
    
    for h in range(len(hubsToBeChanged)):
        nh = []
        for n in nodes:
            # if node is in the same cluster as hub to be changed but is
            # not the hub node itself
            if n.id != hubsToBeChanged[h] and n.hubID == hubsToBeChanged[h]:
                # could be a new hub
                nh.append(n.id)
        new_hubs.append(nh)
                
    # go through list of possible hubs for each hub to be changed
    for i in range(len(new_hubs)):
        hub = -1
        proximity = 100
        for h in new_hubs[i]:
            temp = 0
            for h2 in new_hubs[i]:
                temp += dist(nodes[h].location, nodes[h2].location, 2)
            temp += dist(nodes[h].location, nodes[hubsToBeChanged[i]].location, 2)
            if temp < proximity:
                if nodes[h].energy >= (init_energy * packet_size * nodes[h].transmit_pwr / 1000):
                    proximity = temp
                    hub = h
        # node closest to all others and is not the previous hub will become new hub
        if hub != -1:
            nodes[hubsToBeChanged[i]].setHub(hub)
            for h in new_hubs[i]:
                nodes[h].setHub(hub)
            updated_hubs.append(hub)
        else:
            updated_hubs.append(hubsToBeChanged[i])
        
    return updated_hubs
                
# destination / gateway location
dest = [5, 6]

# initial energy for each node
init_energy = 10

# sensor nodes - initialized with node locations and they'll
# all start with same amount of energy
n0 = Node(0, 10, [0, 2])
n1 = Node(1, 10, [1, 2])
n2 = Node(2, 10, [4, 2])
n3 = Node(3, 10, [3, 1])
n4 = Node(4, 10, [2, 0])

# list of nodes so it's easy to pass them to a function
nodes = [n0, n1, n2, n3, n4]

# for simplicity, packet size is set at 512
packet_size = 512

# determine how many clusters want for the network to start
k = round(len(nodes) / 3)

# create initial network layout
layout(nodes, dest, k)

# start sending packets
total_packets = 0

# while the network is still able to send another packet
continueFlag = True
while continueFlag:
    
    # make sure power for all nodes is good, if good then we can use it
    if not checkNodeEnergy(nodes, packet_size):
        continueFlag = False
    
    # randomly generate source node
    s = random.randint(0, len(nodes)-1)
    if s == nodes[s].hubID:
        if nodes[s].energy >= (packet_size * (nodes[s].transmit_pwr / 1000)):
            total_packets += sendPacket(nodes, s, dest, packet_size)
    elif s != nodes[s].hubID:
        
        total_packets += sendPacket(nodes, s, dest, packet_size)
       
    # check on hub energy levels
    hubsToBeChanged = checkHubsAvailability(nodes, packet_size, init_energy)
    
    if len(hubsToBeChanged) > 0:
        updated_hubs = changeHubs(nodes, hubsToBeChanged, init_energy)
        # if any of the hubs weren't able to be changed
        if np.any(updated_hubs == hubsToBeChanged):
            # change clusters
            if k-1 > 0:
                layout(nodes, dest, k-1)
                k -= 1
            # or make every node in the cluster into a hub so that it is now
            # just a mesh network
            else:
                for i in range(len(updated_hubs)):
                    if updated_hubs[i] == hubsToBeChanged[i]:
                        for n in nodes:
                            if n.hubID == updated_hubs[i]:
                                n.setHub(n.id)
            
print("NETWORK IS UNABLE TO CONTINUE SENDING PACKETS")
print ("NUMBER OF PACKETS SENT: ", total_packets)
for i in range(0, len(nodes)):
    print("n" + str(i) + " energy left: " + str(nodes[i].energy))
    print("how much needed: " + str(packet_size * nodes[i].transmit_pwr / 1000))

NETWORK IS UNABLE TO CONTINUE SENDING PACKETS
NUMBER OF PACKETS SENT:  4
n0 energy left: 6.416
how much needed: 7.7824
n1 energy left: 4.6752
how much needed: 5.3248
n2 energy left: 4.6752
how much needed: 5.3248
n3 energy left: 3.1391999999999998
how much needed: 4.9152
n4 energy left: 3.0368000000000004
how much needed: 6.9632
